In [ ]:
BASE_FOLDER = "HistoQuant/SDH_classify/"

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

In [ ]:
import glob
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.image import resize_with_crop_or_pad

def generate_data_arrays(BASE_FOLDER):
    n_train = len(glob.glob(BASE_FOLDER+"train/*.tif"))
    n_val = len(glob.glob(BASE_FOLDER+"validation/*.tif"))
    n_test = len(glob.glob(BASE_FOLDER+"test/*.tif"))
    counter = 0
    train_images = np.empty(shape=(n_train, 224, 224, 3), dtype=np.float32)
    train_labels = np.empty(shape=(n_train, 1), dtype=np.int8)
    val_images = np.empty(shape=(n_val, 224, 224, 3), dtype=np.float32)
    val_labels = np.empty(shape=(n_val, 1), dtype=np.int8)
    test_images = np.empty(shape=(n_test, 224, 224, 3), dtype=np.float32)
    test_labels = np.empty(shape=(n_test, 1), dtype=np.int8)

    for image in glob.glob(BASE_FOLDER+"train/*.tif"):
        im = np.asarray(Image.open(image))
        im_resized = resize_with_crop_or_pad(im, 224, 224)
        train_images[counter] = im_resized
        train_labels[counter] = int(image.split("/")[-1].split("_")[0])
        counter += 1
    
    counter = 0
    for image in glob.glob(BASE_FOLDER+"validation/*.tif"):
        im = np.asarray(Image.open(image))
        im_resized = resize_with_crop_or_pad(im, 224, 224)
        val_images[counter] = im_resized
        val_labels[counter] = int(image.split("/")[-1].split("_")[0])
        counter += 1
    
    counter = 0
    for image in glob.glob(BASE_FOLDER+"test/*.tif"):
        im = np.asarray(Image.open(image))
        im_resized = resize_with_crop_or_pad(im, 224, 224)
        test_images[counter] = im_resized
        test_labels[counter] = int(image.split("/")[-1].split("_")[0])
        counter += 1

    return train_images, train_labels, val_images, val_labels, test_images, test_labels

In [ ]:
# model = ResNet50(weights='imagenet')
# model.summary()
train_images, train_labels, val_images, val_labels, test_images, test_labels = generate_data_arrays(BASE_FOLDER)

In [ ]:
train_images, val_images, test_images = train_images/255.0, val_images/255.0, test_images/255.0

In [ ]:
class_weights_numpy = np.unique(train_labels, return_counts=True)
n_train = len(train_labels)
class_weights = {class_weights_numpy[0][0]: class_weights_numpy[1][0]/n_train, class_weights_numpy[0][1]: class_weights_numpy[1][1]/n_train}

In [ ]:
import matplotlib.pyplot as plt
class_names = ['Healthy', 'Sick']

plt.figure(figsize=(10,10))
counter = 0
for i in np.random.choice(range(len(train_images)),25):
    plt.subplot(5,5,counter+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i])
    # The CIFAR labels happen to be arrays, 
    # which is why you need the extra index
    plt.xlabel(class_names[train_labels[i][0]])
    counter +=1
plt.show()


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2))
model.summary()

In [ ]:
epochs = 200

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=epochs, 
                    validation_data=(val_images, val_labels), shuffle=True, class_weights=class_weights)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()


In [ ]:
print(val_acc)

In [ ]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print("Test data results: ")
print(test_acc)